In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.neighbors import KNeighborsClassifier
from random import randint, random

In [3]:
# load dataset
df = pd.read_csv("Features_os2.csv")
df

,FCz_1,FCz_2,FCz_3,FCz_4,FCz_5,FCz_6,FCz_7,FCz_8,FCz_9,FCz_10,...,F4_9,F4_10,F4_11,F4_12,F4_13,F4_14,F4_15,F4_16,F4_17,successful
0,243,46.275412,5.251169,46.275412,5.251169,1745.321402,-1556.176982,189.144419,189.144419,3301.498384,...,119.243056,2325.212678,146.464835,65.324283,43,1.519169,1,0.023256,11,0
1,129,19.456415,6.630204,19.456415,6.630204,1168.668172,-1187.517318,-18.849146,18.849146,2356.185490,...,140.883665,1641.550361,118.847656,32.384543,-81,-0.399809,1,-0.012346,12,0
2,207,16.944689,12.216217,16.944689,12.216217,1121.418385,-825.133417,296.284968,296.284968,1946.551802,...,221.432778,1300.504814,111.125225,25.493032,42,0.606977,1,0.023810,9,0
3,58,13.011041,4.457752,13.011041,4.457752,840.795108,-662.270062,178.525046,178.525046,1503.065170,...,54.978582,978.088906,102.282048,22.282922,-101,-0.220623,3,-0.029703,13,0
4,94,19.927048,4.717207,19.927048,4.717207,830.918572,-1137.546198,-306.627626,306.627626,1968.464771,...,275.621429,1527.615153,119.347146,31.960993,98,0.326133,1,0.010204,15,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
239,143,17.228363,8.300266,17.228363,8.300266,620.075000,-955.806666,-335.731666,335.731666,1575.881666,...,335.731666,1575.881666,166.325945,33.700049,100,0.337000,5,0.050000,12,1
240,143,14.112356,10.132964,14.112356,10.132964,695.755618,-778.348484,-82.592866,82.592866,1474.104102,...,82.592866,1474.104102,102.511463,24.141438,58,0.416232,1,0.017241,13,1
241,241,30.494556,7.903050,30.494556,7.903050,2083.810501,-2096.629020,-12.818519,12.818519,4180.439522,...,12.818519,4180.439522,270.078767,101.200611,60,1.686677,1,0.016667,10,1
242,123,21.118092,5.824390,21.118092,5.824390,581.398448,-978.756696,-397.358249,397.358249,1560.155144,...,397.358249,1560.155144,148.303983,34.816309,66,0.527520,1,0.015152,13,1


In [4]:
# split data to 2 groups
err_df = df[df['successful'] == 0]
succ_df = df[df['successful'] == 1]

# create features array and class array for errors
X_err = np.array(err_df.drop(['successful'],1))
y_err = np.array(err_df['successful'])

# create features array and class array for successful clicks
X_succ = np.array(succ_df.drop(['successful'],1))
y_succ = np.array(succ_df['successful'])

# 20% do ostatecznego testu, wyrównana liczba przypadków w grupach
X_err_rest, X_err_end_test, y_err_rest, y_err_end_test = train_test_split(X_err, y_err, test_size=0.2, random_state=3)
X_succ_rest, X_succ_end_test, y_succ_rest, y_succ_end_test = train_test_split(X_succ, y_succ,
                                                                              test_size=y_err_end_test.shape[0],
                                                                              train_size=y_err_rest.shape[0],
                                                                              random_state=3)
# w tym momencie dane do ostatecznego testu wychodzą posegregowane, najpierw błędy interfejsu, potem udane kliknięcia
# w k-NN to nie będzie miało znaczenia, a co w innych klasyfikatorach?
X_end_test = np.concatenate((X_err_end_test, X_succ_end_test))
y_end_test = np.concatenate((y_err_end_test, y_succ_end_test))

# sklejam pozostałe 80% które wykorzystam przy cross-walidacji
X_rest = np.concatenate((X_err_rest, X_succ_rest))
y_rest = np.concatenate((y_err_rest, y_succ_rest))

In [5]:
# pomyłki interfejsu to tylko około 7% danych
# czy powinnam wszystko robić na wyrównanej liczbie przypadków dla obu klas? TAK
# czy to wszystko będzie lepiej działać na arrayach niż na listach? TAK

# rzeczy do przetestowania
# tworząc pierwsze pokolenie osobników zrobić tak żeby od razu miały około tylko 10% (kilkanaście) jedynek
# inne funkcje oceny

# czy w ostatecznym teście danymi uczącymi powinny być wszystkie pozostałe poza testem?
# nie, zrób 10-krotną cross val na tych 10 wierszach

In [6]:
# parametry algorytmu

generation_size = 30
num_of_generations = 200
crossover_prob = 1
mutation_prob = 0.02

In [7]:
# funkcja tworząca pierwsze pokolenie
# jedynki oznaczają cechy sygnału przekazywane do klasyfikatora

def first_gen(generation_size):
    first_gen = []
    for one in range(generation_size):
        ind = []
        for i in range(136):
            ind.append(randint(0, 1))
        first_gen.append(ind)
    return first_gen

In [8]:
# funkcja oceniająca przystosowanie danego osobnika

def eval(individual):
    features = []
    n_of_ones = 0
    for i in range(136):
        if individual[i] == 1:
            features.append(i)
    neigh = KNeighborsClassifier(n_neighbors=3)
    neigh.fit(X_train[:, features], y_train)
    accuracy = neigh.score(X_test[:, features], y_test)
    
#     print(136-len(features), accuracy * (136-len(features))-50)
    
    return 136-len(features), accuracy, accuracy * (136-len(features)) - 50

In [9]:
# funkcja oceniająca całe pokolenie

def evaluation(generation):
    local_best_fitness = 0
    local_best_accuracy = 0
    local_n_of_zeros = 0
    best_individual = []
    gen_fitness = []
    for one in generation:
        n_of_zeros, accuracy, fitness = eval(one)
        if fitness > local_best_fitness:
            local_best_fitness = fitness
            local_best_accuracy = accuracy
            local_n_of_zeros = n_of_zeros
            best_individual = one
        gen_fitness.append(fitness)
    return gen_fitness, local_n_of_zeros, local_best_fitness, local_best_accuracy, best_individual

In [10]:
# ruletka

def roulette(generation, gen_fitness):
    roulette_tab = []
    prev = 0
    for one in gen_fitness:
        prev = one/sum(gen_fitness) + prev
        roulette_tab.append(prev)
    roulette_tab[-1] = 1
    survivors = []
    for i in range(generation_size):
        random_num = random()
        winner = 0
        while random_num > roulette_tab[winner]:
            winner += 1
        survivors.append(generation[winner])
    return survivors

In [11]:
# funkcja krzyżująca osobniki

def crossover(survivors):
    descendants = []
    for i in range(int(generation_size/2)):
        if random() <= crossover_prob:
            cut = randint(1,135)
            i *= 2
            descendants.append(survivors[i][:cut] + survivors[i+1][cut:])
            descendants.append(survivors[i+1][:cut] + survivors[i][cut:])
        else:
            descendants.append(survivors[i])
            descendants.append(survivors[i+1])
    return descendants

In [12]:
# funkcja mutująca wybrane geny wybranych osobników z pokolenia

def mutation(descendants, mutation_prob):
    for one in descendants:
        if random() <= mutation_prob:
            locus = randint(0,135)
#             zamiana genu
            one[locus] = 1 - one[locus]
    return descendants

In [13]:
def main(generation_size, num_of_generations, mutation_prob, crossover_prob):
    
#     inicjalizacja populacji (first generation)
    generation = first_gen(generation_size)
#     print('gen 0')
    
#     ocena przystosowania populacji
    global_best_fitness = 0
    gen_fitness, local_n_of_zeros, local_best_fitness, local_best_accuracy, best_individual = evaluation(generation)
#     print('local ',local_best_fitness)
#     print('num of zeros ', local_n_of_zeros)
    global_best_fitness = local_best_fitness
    global_best_accuracy = local_best_accuracy
#     print(global_best_fitness)
        
#     główna pętla programu
    for i in range(num_of_generations):
#         print('gen ', i+1)   
#     ruletka
        survivors = roulette(generation, gen_fitness)
#     krzyżowanie
        descendants = crossover(survivors)  
#     mutacje
        mutation(descendants, mutation_prob)   
#     ocena
        gen_fitness, local_n_of_zeros, local_best_fitness, local_best_accuracy, local_best_individual = evaluation(descendants)
#         print('local ',local_best_fitness)
#         print('num of zeros ', local_n_of_zeros)
        if local_best_fitness > global_best_fitness:
            global_best_fitness = local_best_fitness
            global_best_accuracy = local_best_accuracy
            best_individual = local_best_individual
#         print(global_best_fitness)
        generation = descendants
    return global_best_fitness, global_best_accuracy, best_individual

In [14]:
# z pozostałych 80% do treningu:
# podział na 5 grup
kf = KFold(n_splits=5, shuffle=True, random_state=7)
alltime_best_fitness = 0
alltime_best_accuracy = 0
best_individual = []

for train_index, test_index in kf.split(X_rest):
    X_train, X_test = X_rest[train_index], X_rest[test_index]
    y_train, y_test = y_rest[train_index], y_rest[test_index]
    global_best_fitness, global_best_accuracy, best_individual = main(generation_size, num_of_generations, 
                                                                      mutation_prob, crossover_prob)
    print(global_best_fitness,"\n", global_best_accuracy, "\n", best_individual)
    if global_best_fitness > alltime_best_fitness:
        alltime_best_fitness = global_best_fitness
        alltime_best_accuracy = global_best_accuracy
        alltime_best_individual = best_individual

39.0 
 1.0 
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0]
53.0 
 1.0 
 [0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0]
0 
 0 
 []
37.0 
 1.0 
 [1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1,

In [15]:
print(alltime_best_fitness,"\n", alltime_best_accuracy, "\n", alltime_best_individual)

53.0 
 1.0 
 [0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0]


In [17]:
features = []
for i in range(136):
    if alltime_best_individual[i] == 1:
        features.append(i)

kf2 = KFold(n_splits=8)

# for train_index, test_index in kf.split(X_end_test):
#     X_train, X_test = X_end_test[train_index], X_end_test[test_index]
#     y_train, y_test = y_end_test[train_index], y_end_test[test_index]
# y_end_test[features]

X = X_end_test[:, features]
y =  y_end_test
neigh3 = KNeighborsClassifier(n_neighbors=3)
score = cross_val_score(neigh3, X, y, cv=kf2)
print(score)
print(score.mean())

[0. 0. 0. 1. 0. 0. 0. 0.]
0.125


In [41]:
# global_best_fitness, global_best_accuracy, best_individual = main(generation_size, num_of_generations, mutation_prob, crossover_prob)

In [16]:
# print(global_best_fitness,"\n", global_best_accuracy, "\n", best_individual)